In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

# from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


Using TensorFlow backend.


In [3]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [4]:
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###
    return X

In [5]:
# def ResNet50(input_shape=(64, 64, 3), classes=6):

def ResNet50(input_shape=(100, 100, 3), classes=6):    
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model


In [6]:
# model = ResNet50(input_shape=(64, 64, 3), classes=6)
model = ResNet50(input_shape=(100, 100, 3), classes=6)

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# A module for creating datasets from reading images from folder'9' under 'dir_9'

import matplotlib.image as mpimg
from PIL import Image
import numpy as np
import os.path

def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = mpimg.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    images_size = np.shape(images)        
    return images_size, images


In [9]:
def label_data_find(digit_quan, one_hot):
    
    label_data_0 = []
    for i in range(digit_quan[0]):
        label_start_0 = one_hot[0]
        label_data_0.append(label_start_0)
        
    label_data_1 = []
    for i in range(digit_quan[1]):
        label_start_1 = one_hot[1]
        label_data_1.append(label_start_1)
        
    label_data_2 = []
    for i in range(digit_quan[2]):
        label_start_2 = one_hot[2]
        label_data_2.append(label_start_2)
        
    label_data_3 = []
    for i in range(digit_quan[3]):
        label_start_3 = one_hot[3]
        label_data_3.append(label_start_3)
        
    label_data_4 = []
    for i in range(digit_quan[4]):
        label_start_4 = one_hot[4]
        label_data_4.append(label_start_4)
        
    label_data_5 = []
    for i in range(digit_quan[5]):
        label_start_5 = one_hot[5]
        label_data_5.append(label_start_5)
        
    label_data_6 = []
    for i in range(digit_quan[6]):
        label_start_6 = one_hot[3]
        label_data_6.append(label_start_6)
        
    label_data_8 = []
    for i in range(digit_quan[7]):
        label_start_8 = one_hot[3]
        label_data_8.append(label_start_8)
        
    label_data_9 = []
    for i in range(digit_quan[8]):
        label_start_9 = one_hot[3]
        label_data_9.append(label_start_9)
        
    return label_data_0, label_data_1, label_data_2, label_data_3, label_data_4, label_data_5, label_data_6, label_data_8, label_data_9


In [10]:
def one_hot_matrix(labels, C):
    """
    Creates a matrix where the i-th row corresponds to the ith class number and the jth column
                     corresponds to the jth training example. So if example j had a label i. Then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the labels 
    C -- number of classes, the depth of the one hot dimension
    
    Returns: 
    one_hot -- one hot matrix
    """
    
    ### START CODE HERE ###
    
    # Create a tf.constant equal to C (depth), name it 'C'. (approx. 1 line)
    C = tf.constant(C, name='C')
    
    # Use tf.one_hot, be careful with the axis (approx. 1 line)
    one_hot_matrix = tf.one_hot(indices=labels, depth=C, axis=0)
    
    # Create the session (approx. 1 line)
    sess = tf.Session()
    
    # Run the session (approx. 1 line)
    one_hot = sess.run(one_hot_matrix)
    
    # Close the session (approx. 1 line). See method 1 above.
    sess.close()
    
    ### END CODE HERE ###
    
    return one_hot

In [11]:
# Creating Y data  
import tensorflow as tf

C = 6

one_hot = one_hot_matrix(np.arange(C), C)


In [12]:
# Load Data
# Creating all datasets, folder-wise and then merging to create single dataset of all images

images_size_0, images_0 = load_images("dir_0/0")
images_size_1, images_1 = load_images("dir_1/1")
images_size_2, images_2 = load_images("dir_2/2")
images_size_3, images_3 = load_images("dir_3/3")
images_size_4, images_4 = load_images("dir_4/4")
images_size_5, images_5 = load_images("dir_5/5")
images_size_6, images_6 = load_images("dir_6/6")
# images_size_7, images_7 = load_images("dir_7/7")
images_size_8, images_8 = load_images("dir_8/8")
images_size_9, images_9 = load_images("dir_9/9")

digit_quan = np.array([images_size_0[0], images_size_1[0], images_size_2[0], images_size_3[0], images_size_4[0], images_size_5[0], images_size_6[0], images_size_8[0], images_size_9[0]])

images_all = np.vstack((images_0, images_1, images_2, images_3, images_4, images_5, images_6, images_8, images_9))


label_data_0, label_data_1, label_data_2, label_data_3, label_data_4, label_data_5, label_data_6, label_data_8, label_data_9 = label_data_find(digit_quan, one_hot)

label_data_all = np.vstack((label_data_0, label_data_1, label_data_2, label_data_3, label_data_4, label_data_5, label_data_6, label_data_8, label_data_9))


X_data = images_all
Y_data = label_data_all

# Generating random order of data(X_data) and related labels(Y_data)

idx = np.random.permutation(len(X_data))
X_random_data, Y_random_data = X_data[idx], Y_data[idx]

# Dividing data into train, validation and test sets. 18% and 22% done to make the validation and test
# sizes equal

import sklearn.model_selection as sk

X_train_cv, X_test, Y_train_cv, Y_test = sk.train_test_split(X_random_data, Y_random_data, test_size = 0.18, random_state=42)

X_train, X_cv, Y_train, Y_cv = sk.train_test_split(X_train_cv, Y_train_cv, test_size = 0.22, random_state=42)

# Normalising X_data

X_train_norm = np.true_divide(X_train, 255)
X_cv_norm = np.true_divide(X_cv, 255)
X_test_norm = np.true_divide(X_test, 255)

print("number of training examples = " + str(X_train_norm.shape[0]))
print("number of test examples = " + str(X_cv_norm.shape[0]))
print("X_train shape: " + str(X_train_norm.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_cv_norm.shape))
print("Y_test shape: " + str(Y_cv.shape))



number of training examples = 1186
number of test examples = 335
X_train shape: (1186, 100, 100, 3)
Y_train shape: (1186, 6)
X_test shape: (335, 100, 100, 3)
Y_test shape: (335, 6)


In [13]:
model.fit(X_train_norm, Y_train, epochs = 30, batch_size = 32)

Epoch 1/30
1186/1186 [==============================] - 283s 239ms/step - loss: 3.5065 - acc: 0.3634
Epoch 2/30
1186/1186 [==============================] - 279s 235ms/step - loss: 2.2204 - acc: 0.4435
Epoch 3/30
1186/1186 [==============================] - 280s 236ms/step - loss: 1.2810 - acc: 0.5691
Epoch 4/30
1186/1186 [==============================] - 449s 378ms/step - loss: 0.9235 - acc: 0.6374
Epoch 5/30
1186/1186 [==============================] - 309s 261ms/step - loss: 0.8280 - acc: 0.7209
Epoch 6/30
1186/1186 [==============================] - 291s 245ms/step - loss: 0.6516 - acc: 0.7909
Epoch 7/30
1186/1186 [==============================] - 289s 244ms/step - loss: 0.5975 - acc: 0.8145
Epoch 8/30
1186/1186 [==============================] - 285s 240ms/step - loss: 0.3511 - acc: 0.8803
Epoch 9/30
1186/1186 [==============================] - 288s 243ms/step - loss: 0.2296 - acc: 0.9444
Epoch 10/30
1186/1186 [==============================] - 293s 247ms/step - loss: 0.3631 - a

In [14]:
preds = model.evaluate(X_test_norm, Y_test)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

335/335 [==============================] - 22s 65ms/step
Loss = 0.4391395180972654
Test Accuracy = 0.9074626869230128
